In [133]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch import optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split



data = pd.read_csv("..\employee_survey.csv")
# data3 = pd.get_dummies(data)

### Exploration

In [66]:
# Data Prep
data.head()

,EmpID,Gender,Age,MaritalStatus,JobLevel,Experience,Dept,EmpType,WLB,WorkEnv,...,SleepHours,CommuteMode,CommuteDistance,NumCompanies,TeamSize,NumReports,EduLevel,haveOT,TrainingHoursPerYear,JobSatisfaction
0,6,Male,32,Married,Mid,7,IT,Full-Time,1,1,...,7.6,Car,20,3,12,0,Bachelor,True,33.5,5
1,11,Female,34,Married,Mid,12,Finance,Full-Time,1,1,...,7.9,Car,15,4,11,0,Bachelor,False,36.0,5
2,33,Female,23,Single,Intern/Fresher,1,Marketing,Full-Time,2,4,...,6.5,Motorbike,17,0,30,0,Bachelor,True,10.5,5
3,20,Female,29,Married,Junior,6,IT,Contract,2,2,...,7.5,Public Transport,13,2,9,0,Bachelor,True,23.0,5
4,28,Other,23,Single,Junior,1,Sales,Part-Time,3,1,...,4.9,Car,20,0,7,0,Bachelor,False,20.5,5


In [92]:
cols = data.select_dtypes(include=['object', 'category']).columns
for col in cols:
    print(col)
    print(data[col].unique())

def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 


Gender
['Male' 'Female' 'Other']
MaritalStatus
['Married' 'Single' 'Divorced' 'Widowed']
JobLevel
['Mid' 'Intern/Fresher' 'Junior' 'Senior' 'Lead']
Dept
['IT' 'Finance' 'Marketing' 'Sales' 'Operations' 'Customer Service'
 'Legal' 'HR']
EmpType
['Full-Time' 'Contract' 'Part-Time']
CommuteMode
['Car' 'Motorbike' 'Public Transport' 'Bike' 'Walk']
EduLevel
['Bachelor' 'High School' 'Master' 'PhD']


### Preprocess

In [ ]:
# Nominal: Gender, Marital Status, Dept, EmpType, CommuteMode
# Ordinal: JobLevel, EduLevel
data2 = data.copy()
data2.drop(columns=['EmpID'], inplace=True)
le = LabelEncoder() 
ohe = OneHotEncoder()

# Ordinal
EduLevel_map = {'High School':1, 'Bachelor':2, 'Master':3, 'PhD':4}
JobLevel_map ={'Intern/Fresher':1, 'Junior':2, 'Mid':3, 'Senior':4, 'Lead':5}
data2['EduLevel'] = data2['EduLevel'].replace(EduLevel_map)
data2['JobLevel'] = data2['JobLevel'].replace(JobLevel_map)

# Nominal
Nominal = ['Gender', 'MaritalStatus', 'Dept', 'EmpType', 'CommuteMode']
for feature in Nominal:
    data2 = encode_and_bind(data2, feature)

# Convert TrueFalse to int
data2['haveOT'] = data2['haveOT'].astype(int)

# Generate X and Y
X = data2.drop(columns=['JobSatisfaction'])
y = data2['JobSatisfaction']

scaler = StandardScaler()
Xscaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(Xscaled, y, test_size=0.2)

# Convert to tensors
X_train_tensor = torch.tensor(X_train.toarray() if hasattr(X_train, "toarray") else X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.toarray() if hasattr(X_test, "toarray") else X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

print(X_train.shape)
print(y_train.shape)

(2420, 39)
(2420,)
(2420, 39)
(2420,)


### Model Design

In [ ]:
# device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # self.flatten = nn.Flatten()
        self.model = nn.Sequential(
            nn.Linear(39, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.model(x)

model = NeuralNetwork()
print(model)

NeuralNetwork(
  (model): Sequential(
    (0): Linear(in_features=39, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
  )
)


### Training

In [128]:
batch_size = 64

# Create data loaders.
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

for X, y in test_loader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 39])
Shape of y: torch.Size([64, 1]) torch.float32


In [130]:
# Code from Pytorch Quickstart 
# https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            # X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [134]:
# Setting up problem
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training Loop
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer)
    test(test_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 10.802662  [   64/ 2420]
Test Error: 
 Accuracy: 0.0%, Avg loss: 2.828306 

Epoch 2
-------------------------------
loss: 2.421798  [   64/ 2420]
Test Error: 
 Accuracy: 0.0%, Avg loss: 1.459365 

Epoch 3
-------------------------------
loss: 1.580099  [   64/ 2420]
Test Error: 
 Accuracy: 0.0%, Avg loss: 1.297346 

Epoch 4
-------------------------------
loss: 1.555732  [   64/ 2420]
Test Error: 
 Accuracy: 0.0%, Avg loss: 1.255004 

Epoch 5
-------------------------------
loss: 1.518747  [   64/ 2420]
Test Error: 
 Accuracy: 0.0%, Avg loss: 1.237876 

Done!
